In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
import re
from dateutil.parser import parse
import string
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
import missingno as msno
from plotly.subplots import make_subplots
from sklearn.preprocessing import MinMaxScaler
import scipy
import researchpy as rp
from matplotlib.offsetbox import AnchoredText
import pacmap
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
import pickle
import pmdarima as pm
from ipywidgets import Button, HBox, VBox
from ipywidgets import interactive
import ipywidgets
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split 
import xgboost as xgb
from sklearn.metrics import mean_squared_error
sns.set_style("darkgrid")
pd.set_option('display.max_columns', None)
pd.options.display.max_colwidth = 100

In [2]:
# Select first of vector of atollos cases (0) and append into DF:
def inicio_perdida_tph(DF, variable):
       first_case_atollo = [0]
       for x in range(1, len(DF)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x-1], variable] == 0)):
                     first_case_atollo.append(1)
              else:
                     first_case_atollo.append(0)
       return first_case_atollo
       
# Select last of vector of atollos cases (0) and append into DF:
def final_perdida_tph(DF, variable):
       last_case_atollo = []
       for x in range((len(DF)-1)):
              if((DF.loc[DF.index[x], variable] == 1) & (DF.loc[DF.index[x+1], variable] == 0)):
                     last_case_atollo.append(1)
              else:
                     last_case_atollo.append(0)
       last_case_atollo.append(1)              
       return last_case_atollo

# Create function Atollo_mod: 0 when initial atollo; 2 when not initial atollo; 1 when no atollo:
def loss_tph(DF,variable):
    
    DF['start loss tph']=inicio_perdida_tph(DF=DF, variable=variable)
    DF['end loss tph']=final_perdida_tph(DF=DF, variable=variable)
    #Create Atollo_mod:
    variable_mod = str(variable+'_mod')    
    DF[variable_mod] =  DF[variable].replace(1,2)   
    DF.loc[DF['start loss tph']==1,variable_mod]=1
    return DF

In [64]:
# read df full csv
df_full=pd.read_csv('../../data/consolidated data/df_full_01November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')["2020":"2021"]
df_full.head()

,sag mill expert system,Limitado por Límite Alto de Tonelaje,Optimizando tonelaje,Limitado por Peso,Limitado por Corriente Bobina,Limitado por torque espesador 2,Limitado por torque espesador 3,Limitado por Potencia Bolas 1,Limitado por Potencia Bolas 2,Limitado por Pebbles,Limitado por Corriente harnero,Limitado por Potencia SAG,Limitado por limite bajo de tonelaje,Limitado por Temperatura bobina,Limitado por presión espesador 2,Limitado por presión espesador 3,pyrite law,chalcopyrite law,chalcocite law,covelin law,crusher index,sag power index,ball work index,bornite law,torque,charge cell,power,TPH,solid percentage,water,speed,solid,LL charge cell,HH charge cell,HH TPH,stator current,granulometry,Edad,dif_TPH_HH,dif_CC_HH,dif_CC_LL,loss of TPH,cluster,recommendation,loss of TPH_1,loss of TPH_2,loss of TPH_3,loss of TPH_4,loss of TPH_5,greater than HH CC,greater than HH CC_1,greater than HH CC_2,greater than HH CC_3,greater than HH CC_4,greater than HH CC_5,less than LL CC,less than LL CC_1,less than LL CC_2,less than LL CC_3,less than LL CC_4,less than LL CC_5,greater than Rec HH CC,greater than Rec HH CC_1,greater than Rec HH CC_2,greater than Rec HH CC_3,greater than Rec HH CC_4,greater than Rec HH CC_5,Rec HH CC less than HH CC,Rec HH CC less than HH CC_1,Rec HH CC less than HH CC_2,Rec HH CC less than HH CC_3,Rec HH CC less than HH CC_4,Rec HH CC less than HH CC_5,start loss tph,end loss tph,loss of TPH_mod
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-01-10 04:05:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.384706,809.237330,22008.766259,4317.030837,73.888038,1539.203093,9.063024,73.155012,770.0,850.0,3950.0,3755.398344,48.829422,1.0,-367.030837,40.762670,39.237330,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
2020-01-10 04:06:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.333379,808.519162,21990.247202,4053.372184,72.953594,1488.284682,9.062035,73.403343,770.0,850.0,3950.0,3748.480419,48.633147,1.0,-103.372184,41.480838,38.519162,0,2,892.044136,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
2020-01-10 04:07:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.204651,803.154714,21947.056756,4087.871015,73.389467,1484.665416,9.063110,73.466667,770.0,850.0,3950.0,3741.271443,48.493040,1.0,-137.871015,46.845286,33.154714,0,2,879.670845,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
2020-01-10 04:08:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.199681,800.856144,21870.036059,3995.961643,73.140566,1461.269967,9.017207,73.585003,770.0,850.0,3950.0,3736.261062,48.465917,1.0,-45.961643,49.143856,30.856144,0,2,879.670845,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0
2020-01-10 04:09:00,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.3,1.76,0.5,0.02,19.7649,98.5569,15.4504,0.05,64.143161,798.234539,21777.881944,4019.595784,73.532265,1443.665449,9.000981,73.910026,770.0,850.0,3950.0,3730.068618,48.495837,1.0,-69.595784,51.765461,28.234539,0,2,879.670845,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0,0,0


In [65]:
len(df_full[df_full["start loss tph"]==1].index)

4438

# Indentificación de caidas de TPH

In [6]:
list_df=[]
for i in df_full[df_full["start loss tph"]==1].index:
    try:
        #i="2022-02-27 12:31:00"
        inicio_perdida=i
        test=df_full[inicio_perdida:inicio_perdida+timedelta(hours=5)]
        fin_perdida=min(test[(test["end loss tph"]==1)].index)+timedelta(minutes=1)
        inicio_evento=inicio_perdida-timedelta(minutes=5)
        data=df_full.copy()[inicio_evento:fin_perdida]
        #display(data)
        dic_df={"inicio evento":inicio_evento,"inicio perdida":inicio_perdida,"fin perdida":fin_perdida,"variación HH TPH":data["HH TPH"].std()!=0,"Actua sistema de control":any(data["greater than HH CC"]==1),
        "Supera recomendación del modelo":any(data["greater than Rec HH CC"]==1),"Recomendación menor a HH CC":any(data["Rec HH CC less than HH CC"]==1),"CC menor a LL CC":any(data["less than LL CC"]==1),
        "Causalidad":any(data[data["greater than Rec HH CC"]==1].index<=inicio_perdida)}
        list_df.append(pd.DataFrame(dic_df,index=[1]))  
    except:
        #print(i)
        pass

In [7]:
df_tph_loss=pd.concat(list_df).reset_index(drop=True)
df_tph_loss.head()

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2021-12-01 00:20:00,2021-12-01 00:25:00,2021-12-01 00:57:00,False,False,False,True,False,False
1,2021-12-01 02:27:00,2021-12-01 02:32:00,2021-12-01 02:40:00,True,False,False,True,False,False
2,2021-12-01 03:17:00,2021-12-01 03:22:00,2021-12-01 03:28:00,True,False,False,True,False,False
3,2021-12-01 04:16:00,2021-12-01 04:21:00,2021-12-01 04:33:00,True,True,True,True,False,True
4,2021-12-01 05:15:00,2021-12-01 05:20:00,2021-12-01 06:28:00,True,True,True,True,True,True


In [66]:
# Se guarda en csv
#df_tph_loss.to_csv('df_tph_loss.csv', index=False)
df_tph_loss=pd.read_csv("df_tph_loss.csv")
df_tph_loss

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-01-10 05:23:00,2020-01-10 05:28:00,2020-01-10 05:35:00,True,False,False,False,False,False
1,2020-01-10 07:05:00,2020-01-10 07:10:00,2020-01-10 08:41:00,False,True,False,False,False,False
2,2020-01-10 09:05:00,2020-01-10 09:10:00,2020-01-10 10:16:00,False,True,False,False,False,False
3,2020-01-10 10:15:00,2020-01-10 10:20:00,2020-01-10 11:08:00,False,True,True,True,False,False
4,2020-01-14 03:54:00,2020-01-14 03:59:00,2020-01-14 04:31:00,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...
4528,2022-02-27 08:01:00,2022-02-27 08:06:00,2022-02-27 09:19:00,False,True,True,True,False,True
4529,2022-02-27 09:42:00,2022-02-27 09:47:00,2022-02-27 12:49:00,False,True,True,True,False,True
4530,2022-02-27 12:45:00,2022-02-27 12:50:00,2022-02-27 12:58:00,False,False,True,True,False,True
4531,2022-02-28 06:51:00,2022-02-28 06:56:00,2022-02-28 07:03:00,False,False,True,True,False,False


# Modelo predictivo TPH

In [9]:
# read df full csv
df_features=pd.read_csv('../../data/consolidated data/df_features_05November2022.csv', parse_dates=['Timestamp'], index_col='Timestamp')["2020":"2021"]
df_features["TPH"]=df_features["TPH"].shift(-5)
df_features["loss of TPH"]=df_features["loss of TPH"].shift(-5)
df_features.dropna(inplace=True)
df_features.sort_index(inplace=True)
df_features.shape

(796472, 340)

In [10]:
# Generación de targets ficticio y features
target=df_features["2020":"2021-11"]["loss of TPH"].copy()
features=df_features["2020":"2021-11"].drop(columns=["loss of TPH"])

# Generación conjunto de train y test 
Xtrain, Xtest, Ytrain, Ytest = train_test_split(
    features, target, test_size=0.33, 
    random_state=42
)

# Sobremuestreo: Aumentar numero de datos de la clase minoritaria (perdida de TPH)
# ROS(Duplica clases):
ros=RandomOverSampler(random_state=0) # Random_state=0

# ROS:
Xtrain, Ytrain=ros.fit_resample(Xtrain,Ytrain)

In [11]:
# Ytest reg
Ytest=Xtest["TPH"]
Xtest=Xtest.drop(columns=["TPH"])

In [12]:
# Ytrain reg
Ytrain=Xtrain["TPH"]
Xtrain=Xtrain.drop(columns=["TPH"])

In [13]:
tag_select=['min_water_5',
 'HH TPH',
 'rms_speed_10',
 'rms_sag power index_10',
 'max_charge cell_10',
 'min_granulometry_10',
 'chalcocite law_(t-3)',
 'min_pyrite law_5',
 'max_bornite law_10',
 'Edad',
 'mean_crusher index_3',
 'var_water_10',
 'var_covelin law_5',
 'var_charge cell_10',
 'min_ball work index_10',
 'min_chalcopyrite law_10',
 'covelin law_(t-4)',
 'var_crusher index_3',
 'var_bornite law_10',
 'var_speed_10',
 'var_chalcocite law_3',
 'var_water_5',
 'var_sag power index_3',
 'var_ball work index_3']

In [14]:
# Preprocesamiento de datos

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('MinMax', MinMaxScaler(),
        tag_select
        ),
    ])

pipe = Pipeline(
    [
        ("preprocesamiento", preprocessing_transformer), 
     ("clf", xgb.XGBRegressor(seed=1,eval_metric='rmse',gamma= 0.0001, learning_rate= 0.334, n_estimators= 300
     
     ))
    ]
)
pipe.fit(Xtrain, Ytrain)

Pipeline(steps=[('preprocesamiento',
                 ColumnTransformer(transformers=[('MinMax', MinMaxScaler(),
                                                  ['min_water_5', 'HH TPH',
                                                   'rms_speed_10',
                                                   'rms_sag power index_10',
                                                   'max_charge cell_10',
                                                   'min_granulometry_10',
                                                   'chalcocite law_(t-3)',
                                                   'min_pyrite law_5',
                                                   'max_bornite law_10', 'Edad',
                                                   'mean_crusher index_3',
                                                   'var_water_10',
                                                   'var_covelin law_5',
                                                   'var_charge cell_10',
      

In [15]:
# test 2020
Ypred= pipe.predict(Xtest)
# Métricas
print('R2: ',r2_score(Ytest, Ypred),'\n','RMSE: ',mean_squared_error(Ytest, Ypred, squared=False))

R2:  0.9051879089713314 
 RMSE:  139.00180710875935


In [16]:
# test 2021-12
test_time_serie=df_features["2021-12"]
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)
#test_time_serie["TPH predict"]=test_time_serie["TPH predict"].shift(5)

# Métricas
print('R2: ',r2_score(test_time_serie["TPH"], test_time_serie["TPH predict"]),'\n','RMSE: ',mean_squared_error(test_time_serie["TPH"], test_time_serie["TPH predict"], squared=False))

R2:  0.7599387766172159 
 RMSE:  173.08868942734577


# Visualización

In [67]:
# predict full
test_time_serie=df_features
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)

In [ ]:
df_full["TPH predict"]=test_time_serie["TPH predict"].shift(5)

In [70]:
evento=df_tph_loss.sample(1)
data=df_full.copy()[evento["inicio evento"].iloc[0]:evento["fin perdida"].iloc[0]]
evento

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
1606,2020-11-16 04:09:00,2020-11-16 04:14:00,2020-11-16 05:47:00,False,True,False,False,True,False


In [52]:
data=df_full

In [71]:
titulos=tuple(["TPH & HH TPH", "CC, recommendation HH CC, HH CC & LL CC","granulometry","SPI","loss of TPH","diff. HH TPH between TPH",
"greater than HH CC","greater than Rec HH CC","Rec HH CC less than HH CC","less than LL CC","start loss tph","end loss tph","water","speed","solid percentage"])
fig = make_subplots(
    rows=5, cols=3,
    subplot_titles=titulos,
    #subplot_titles=tuple("TPH y HH TPH,"),
     shared_xaxes=True,
     #shared_yaxes=True
    )


fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                    mode='lines',
                    name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                    mode='lines',
                    name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
                    mode='lines',
                    name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                    mode='lines',
                    name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  



fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], ##FF6511
                    mode='lines',
                    name="LL charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], ##FF6511
                    mode='lines',
                    name="HH charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data["recommendation"], ##FF6511
                    mode='lines',
                    name="recommendation HH CC",line=dict(width=1,color="black"),legendgroup = '1'),row=1, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                    mode='lines',
                    name='granulometry',line=dict(width=3,),legendgroup = '1'),row=1, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                    mode='lines',
                    name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                    mode='lines',
                    name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                    mode='lines',
                    name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=5, col=3)  


for i in range(1,6):
   for j in range(1,4):
        fig.update_xaxes(tickformat="%H:%M",row=i, col=j)


fig.add_trace(go.Scatter(x=data.index, y=data[ 'water'], ##FF6511
                    mode='lines',
                    name= 'water',line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  



fig.add_trace(go.Scatter(x=data.index, y=data[  'loss of TPH'], ##FF6511
                    mode='lines',
                    name=  'loss of TPH',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  


fig.add_trace(go.Scatter(x=data.index, y=data[ "dif_TPH_HH"], ##FF6511
                    mode='lines',
                    name=  "dif_TPH_HH",line=dict(width=3, ),legendgroup = '1'),row=2, col=3)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'less than LL CC'], ##FF6511
                    mode='lines',
                    name=  'less than LL CC',line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  


fig.add_trace(go.Scatter(x=data.index, y=data[ 'start loss tph'], ##FF6511
                    mode='lines',
                    name=  'start loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'end loss tph'], ##FF6511
                    mode='lines',
                    name=  'end loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=3)  

fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than Rec HH CC'], ##FF6511
                    mode='lines',
                    name=  'greater than rec HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  

fig.add_trace(go.Scatter(x=data.index, y=data[ "Rec HH CC less than HH CC"], ##FF6511
                    mode='lines',
                    name=  "Rec HH CC less than HH CC",line=dict(width=3, ),legendgroup = '1'),row=3, col=3)  
fig.update_layout(height=1000, width=1500, title_text="Temporal signal analysis")
fig.update_layout(hovermode="x unified")
#

fig.show()

# Generación de contexto de perdida de TPH

In [79]:
# predict full
test_time_serie=df_features
test_time_serie["TPH predict"]=pipe.predict(test_time_serie)
df_full["TPH predict"]=test_time_serie["TPH predict"].shift(5)

In [76]:
evento=df_tph_loss[(df_tph_loss["Actua sistema de control"]==False) & (df_tph_loss["variación HH TPH"]==False) & (df_tph_loss["CC menor a LL CC"]==False) & (df_tph_loss["Recomendación menor a HH CC"]==True) \
    & (df_tph_loss["Supera recomendación del modelo"]==True)\
    & (df_tph_loss["Causalidad"]==True)].reset_index(drop=True)#.sample(1)
evento

,inicio evento,inicio perdida,fin perdida,variación HH TPH,Actua sistema de control,Supera recomendación del modelo,Recomendación menor a HH CC,CC menor a LL CC,Causalidad
0,2020-03-05 11:24:00,2020-03-05 11:29:00,2020-03-05 11:44:00,False,False,True,True,False,True
1,2020-03-05 15:41:00,2020-03-05 15:46:00,2020-03-05 15:52:00,False,False,True,True,False,True
2,2020-03-10 02:19:00,2020-03-10 02:24:00,2020-03-10 02:32:00,False,False,True,True,False,True
3,2020-03-15 01:35:00,2020-03-15 01:40:00,2020-03-15 02:10:00,False,False,True,True,False,True
4,2020-03-15 13:41:00,2020-03-15 13:46:00,2020-03-15 13:54:00,False,False,True,True,False,True
...,...,...,...,...,...,...,...,...,...
226,2022-02-26 16:42:00,2022-02-26 16:47:00,2022-02-26 16:53:00,False,False,True,True,False,True
227,2022-02-27 04:51:00,2022-02-27 04:56:00,2022-02-27 05:21:00,False,False,True,True,False,True
228,2022-02-27 05:29:00,2022-02-27 05:34:00,2022-02-27 05:40:00,False,False,True,True,False,True
229,2022-02-27 05:46:00,2022-02-27 05:51:00,2022-02-27 06:48:00,False,False,True,True,False,True


In [87]:
dic_images={}
for i in range(len(evento)):
    try:
        data=df_full[evento.iloc[i]["inicio evento"]:evento.iloc[i]["fin perdida"]]
        ini_titulo=pd.to_datetime(evento.iloc[i]["inicio evento"])
        #print(evento.iloc[i]["inicio evento"])
        titulos=tuple(["TPH & HH TPH", "CC, recommendation HH CC, HH CC & LL CC","granulometry","SPI","loss of TPH","diff. HH TPH between TPH",
        "greater than HH CC","greater than Rec HH CC","Rec HH CC less than HH CC","less than LL CC","start loss tph","end loss tph","water","speed","solid percentage"])
        fig = make_subplots(
            rows=5, cols=3,
            subplot_titles=titulos,
            #subplot_titles=tuple("TPH y HH TPH,"),
            #shared_xaxes=True
            )


        fig.add_trace(go.Scatter(x=data.index, y=data['TPH'], ##FF6511
                            mode='lines',
                            name="TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

        fig.add_trace(go.Scatter(x=data.index, y=data['HH TPH'], ##FF6511
                            mode='lines',
                            name="HH TPH",line=dict(width=3, ),legendgroup = '1'),row=1, col=1)  

        fig.add_trace(go.Scatter(x=data.index, y=data["TPH predict"], ##FF6511
                            mode='lines',
                            name="regressor model TPH",line=dict(width=1,color="black" ),legendgroup = '1'),row=1, col=1)  

        fig.add_trace(go.Scatter(x=data.index, y=data['charge cell'], 
                            mode='lines',
                            name="charge cell",line=dict(width=3, ),legendgroup = '1'),row=1, col=2)  



        fig.add_trace(go.Scatter(x=data.index, y=data['LL charge cell'], ##FF6511
                            mode='lines',
                            name="LL charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data['HH charge cell'], ##FF6511
                            mode='lines',
                            name="HH charge cell",line=dict(width=3,),legendgroup = '1'),row=1, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data["recommendation"], ##FF6511
                            mode='lines',
                            name="recommendation HH CC",line=dict(width=1,color="black"),legendgroup = '1'),row=1, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data['granulometry'], ##FF6511
                            mode='lines',
                            name='granulometry',line=dict(width=3,),legendgroup = '1'),row=1, col=3)  


        fig.add_trace(go.Scatter(x=data.index, y=data['sag power index'], ##FF6511
                            mode='lines',
                            name="sag power index",line=dict(width=3, ),legendgroup = '1'),row=2, col=1)  



        fig.add_trace(go.Scatter(x=data.index, y=data['speed'], ##FF6511
                            mode='lines',
                            name="speed",line=dict(width=3, ),legendgroup = '1'),row=5, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data['solid percentage'], ##FF6511
                            mode='lines',
                            name='solid percentage',line=dict(width=3, ),legendgroup = '1'),row=5, col=3)  


        for u in range(1,6):
            for v in range(1,4):
                    fig.update_xaxes(tickformat="%H:%M",row=u, col=v)


        fig.add_trace(go.Scatter(x=data.index, y=data[ 'water'], ##FF6511
                            mode='lines',
                            name= 'water',line=dict(width=3, ),legendgroup = '1'),row=5, col=1)  



        fig.add_trace(go.Scatter(x=data.index, y=data[  'loss of TPH'], ##FF6511
                            mode='lines',
                            name=  'loss of TPH',line=dict(width=3, ),legendgroup = '1'),row=2, col=2)  


        fig.add_trace(go.Scatter(x=data.index, y=data[ "dif_TPH_HH"], ##FF6511
                            mode='lines',
                            name=  "dif_TPH_HH",line=dict(width=3, ),legendgroup = '1'),row=2, col=3)  


        fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than HH CC'], ##FF6511
                            mode='lines',
                            name=  'greater than HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=1)  

        fig.add_trace(go.Scatter(x=data.index, y=data[ 'less than LL CC'], ##FF6511
                            mode='lines',
                            name=  'less than LL CC',line=dict(width=3, ),legendgroup = '1'),row=4, col=1)  


        fig.add_trace(go.Scatter(x=data.index, y=data[ 'start loss tph'], ##FF6511
                            mode='lines',
                            name=  'start loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data[ 'end loss tph'], ##FF6511
                            mode='lines',
                            name=  'end loss tph',line=dict(width=3, ),legendgroup = '1'),row=4, col=3)  

        fig.add_trace(go.Scatter(x=data.index, y=data[ 'greater than Rec HH CC'], ##FF6511
                            mode='lines',
                            name=  'greater than rec HH CC',line=dict(width=3, ),legendgroup = '1'),row=3, col=2)  

        fig.add_trace(go.Scatter(x=data.index, y=data[ "Rec HH CC less than HH CC"], ##FF6511
                            mode='lines',
                            name=  "Rec HH CC less than HH CC",line=dict(width=3, ),legendgroup = '1'),row=3, col=3)  
        fig.update_layout(height=1000, width=1500, title_text=f"Temporal signal analysis - {ini_titulo}")
        fig.update_layout(hovermode="x unified")
        #

        #fig.show()
        fecha=ini_titulo.strftime("%Y-%m-%d")
        dic_images[i]=fig
        fig.write_image(f"../../images/Escenarios de caidas de TPH vs predict a 5 minutos/{i}_fig_{fecha}.png")
        print(i)
    except Exception as e:
        print(i)
        print(e)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
